`--- Day 4: Repose Record ---`

In [ ]:
from collections import Counter
import re

In [ ]:
input_data = open('input.txt', 'r').read().splitlines()

In [ ]:
input_data = sorted(input_data) # yyyy-mm-dd hh:mm format sorts correctly lexically

In [4]:
class Guard(object):
    
    def __init__(self, guard_id):
        self.guard_id = guard_id
        self.counts = Counter()
        self.sleeptime = 0
        self.cached_most_frequent = None # might be premature optimisation... not sure how expensive self.most_frequent() is
        
    def sleep(self, start, end):
        if end <= start:
            raise ValueError('end must be after start!')
        self.counts += Counter(range(start, end))
        self.sleeptime += end - start
        self.cached_most_frequent = None # invalidate cache
        
    def most_frequent(self):
        if self.cached_most_frequent is None:
            most_common = self.counts.most_common(1)
            if len(most_common) == 0: # not yet slept
                maxv = 0
            else:
                maxv = most_common[0][1]
            self.cached_most_frequent = [k for k,v in self.counts.items() if v == maxv]
        return self.cached_most_frequent
        
    def __str__(self):
        return f'Guard #{self.guard_id}: sleeptime = {self.sleeptime}, most_frequent = {self.most_frequent()}'
    
    __repr__ = __str__

In [5]:
def process_records(data):
    guard_on_duty_id = None
    guard_on_duty = None
    
    guard_ids = set()
    guards = {}
    
    sleep_start = None
    
    # The following assumes that a "falls asleep" for a guard is always paired with a corresponding "wakes up"
    # that is true for my input data.  In general, this should allow for a guard falling asleep, but not waking up
    # before the end of the hour (detect that condition and log a sleep of (start_time, 60))
    for l in data:
        if 'Guard' in l:
            guard_on_duty_id = int(re.findall(r'#(\d+)', l)[0])
            if guard_on_duty_id in guard_ids:
                guard_on_duty = guards[guard_on_duty_id]
            else:
                guard_on_duty = Guard(guard_on_duty_id)
                guards[guard_on_duty_id] = guard_on_duty
                guard_ids.add(guard_on_duty_id)
        else:
            minute = int(re.findall(r'\d+', l)[4])
            if 'falls' in l:
                sleep_start = minute
            elif 'wakes' in l:
                guard_on_duty.sleep(sleep_start, minute)
                sleep_start = None
            else:
                raise NotImplementedError(f'unrecognised line \'{l}\'')
    
    return guard_ids, guards

def strategy1(guard_ids, guards):
    max_sleeptime = max(guards[gid].sleeptime for gid in guard_ids)
    sleepiest = [gid for gid in guard_ids if guards[gid].sleeptime == max_sleeptime][0] # assumption that there is only one
    return sleepiest * guards[sleepiest].most_frequent()[0]

# Part 1
## test case

In [6]:
test_data = '''[1518-11-01 00:00] Guard #10 begins shift
[1518-11-01 00:05] falls asleep
[1518-11-01 00:25] wakes up
[1518-11-01 00:30] falls asleep
[1518-11-01 00:55] wakes up
[1518-11-01 23:58] Guard #99 begins shift
[1518-11-02 00:40] falls asleep
[1518-11-02 00:50] wakes up
[1518-11-03 00:05] Guard #10 begins shift
[1518-11-03 00:24] falls asleep
[1518-11-03 00:29] wakes up
[1518-11-04 00:02] Guard #99 begins shift
[1518-11-04 00:36] falls asleep
[1518-11-04 00:46] wakes up
[1518-11-05 00:03] Guard #99 begins shift
[1518-11-05 00:45] falls asleep
[1518-11-05 00:55] wakes up'''.splitlines()

# test case
assert(strategy1(*process_records(test_data)) == 240)

## answer

In [7]:
print(f'Part 1: {strategy1(*process_records(input_data))}')

Part 1: 71748


# Part 2

In [8]:
def strategy2(guard_ids, guards):
    gid, max_frequency, count = max([(gid,
                                      None if len(guards[gid].most_frequent()) == 0 else guards[gid].most_frequent()[0], 
                                      0 if len(guards[gid].most_frequent()) == 0 else guards[gid].counts[guards[gid].most_frequent()[0]])
                                     for gid in guard_ids],
                                    key=lambda x: x[2])
    return gid * max_frequency
    

## test case

In [9]:
assert(strategy2(*process_records(test_data)) == 4455)

## answer

In [10]:
print(f'Part 2: {strategy2(*process_records(input_data))}')

Part 2: 106850
